In [1]:
import tensorflow as tf
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16
from keras.callbacks import ModelCheckpoint

# Define the input shape and number of classes
INPUT_SHAPE = (224, 224, 3)
num_classes = 8
BATCH_SIZE = 32

In [2]:
# Define the paths to the train, validation, and test directories
TRAIN_DIR = "/kaggle/input/preprocessed-dataset/dataset-split/train"
VAL_DIR = "/kaggle/input/preprocessed-dataset/dataset-split/val"
TEST_DIR = "/kaggle/input/preprocessed-dataset/dataset-split/test"

# Define the data augmentation parameters
data_augmentation = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.1
)

# Define the training dataset generator
train_generator = data_augmentation.flow_from_directory(
    directory=TRAIN_DIR,
     target_size=INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

# Define the validation dataset generator
val_generator = ImageDataGenerator().flow_from_directory(
    directory=VAL_DIR,
     target_size=INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

# Define the test dataset generator
test_generator = ImageDataGenerator().flow_from_directory(
    directory=TEST_DIR,
    target_size=INPUT_SHAPE[:2],
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

Found 3960 images belonging to 8 classes.
Found 1130 images belonging to 8 classes.
Found 575 images belonging to 8 classes.


In [3]:
# Load the VGG16 base model and freeze its layers
vgg_base = VGG16(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
vgg_base.trainable = False

# Define the model architecture
model = Sequential()
model.add(vgg_base)
model.add(layers.Reshape((7, 7*512), input_shape=(7, 7, 512)))
model.add(LSTM(units=256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=256, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=128, return_sequences=True))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
print(model.summary())

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 reshape (Reshape)           (None, 7, 3584)           0         
                                                                 
 lstm (LSTM)                 (None, 7, 256)            3933184   
                                                                 
 dropout (Dropout)           (None, 7, 256)            0         
                                                                 
 lstm_1 (LSTM)               (None, 7, 256)            525312    
                                                                 
 dropout_1 (Dropout)         (None, 7, 256)            0         
                                                         

In [4]:
filepath = "/kaggle/working/LSTM.h5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [5]:
# train model
history = model.fit(
    train_generator, 
    steps_per_epoch=train_generator.n // train_generator.batch_size, 
    epochs=50, 
    validation_data=val_generator, 
    validation_steps=val_generator.n // val_generator.batch_size,
    callbacks=callbacks_list)

Epoch 1/50
123/123 [==============================] - ETA: 0s - loss: 2.0081 - accuracy: 0.1853
Epoch 1: loss improved from inf to 2.00812, saving model to /kaggle/working/LSTM.h5
123/123 [==============================] - 116s 816ms/step - loss: 2.0081 - accuracy: 0.1853 - val_loss: 1.9248 - val_accuracy: 0.2402
Epoch 2/50
123/123 [==============================] - ETA: 0s - loss: 1.8783 - accuracy: 0.2480
Epoch 2: loss improved from 2.00812 to 1.87828, saving model to /kaggle/working/LSTM.h5
123/123 [==============================] - 74s 604ms/step - loss: 1.8783 - accuracy: 0.2480 - val_loss: 1.9091 - val_accuracy: 0.2438
Epoch 3/50
123/123 [==============================] - ETA: 0s - loss: 1.8084 - accuracy: 0.2770
Epoch 3: loss improved from 1.87828 to 1.80845, saving model to /kaggle/working/LSTM.h5
123/123 [==============================] - 76s 616ms/step - loss: 1.8084 - accuracy: 0.2770 - val_loss: 1.7751 - val_accuracy: 0.3009
Epoch 4/50
123/123 [=============================

In [6]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator,verbose=2)
print('Test accuracy:', test_acc)
print('Test Loss:', test_loss)

18/18 - 39s - loss: 1.4285 - accuracy: 0.4400 - 39s/epoch - 2s/step
Test accuracy: 0.4399999976158142
Test Loss: 1.428511142730713
